In [1]:
#!pip install pymongo
# uncomment this if pymongo not installed in your environment

In [ ]:
from kafka import KafkaConsumer
import json
import pymongo as mdb

In [ ]:
# Initialize Kafka Consumer
bootstrap_servers = 'kafka:9092'
topic_name = 'nba-API-data'
consumer = KafkaConsumer(topic_name, bootstrap_servers=bootstrap_servers, value_deserializer=lambda x: json.loads(x.decode('utf-8')))

In [ ]:
# Initialize MongoDB client and database
client = mdb.MongoClient("mongodb://pt-n20.p4001.w3.cs.technikum-wien.at:4001")
db = client.nba_data
collection = db.season_stats_api

In [ ]:
for message in consumer:
    data = message.value
    player_id = data['id']
    full_name = data['full_name']
    season = data['season']
    season_stats = data['season_stats']

    # Check if the document already exists
    existing_doc = collection.find_one({
        'player_id': player_id,
        'full_name': full_name,
        'season': season
    })

    if existing_doc is None:
        # Insert the document into MongoDB
        doc = {
            'player_id': player_id,
            'full_name': full_name,
            'season': season,
            'season_stats': season_stats
        }
        collection.insert_one(doc)
        print(f"Inserted {player_id} - Season {season}")
    else:
        print("Skipped")

In [ ]:
consumer.close()

In [ ]:
mongo_client.close()